In [173]:
import numpy as np                   # Numerical computation library
import pandas as pa                  # Dataframe and Data manipulation Library
import matplotlib.pyplot as plt      # Basic plotting functionality Library
import seaborn as sn                 # Advanced Data visualization Library
import numpy.random as rn
from sklearn.linear_model import LinearRegression
import matplotlib.colors as pltco # A package to help with coloring plots


[x for x in dir(list) if not '__' in x]
  
# Curreny Exchange Rates compared to the US dollar from around the world 22 for 2008-2018
gdp_per_industry = pa.read_csv('https://www.dropbox.com/s/emd7111t9g0rtd2/career_projections_group.csv?dl=1')
workforce_demographic2018 = pa.read_csv('https://www.dropbox.com/s/w1zzumsadlgg9pn/Colorado_Current_Workforce_Demographic.csv?dl=1')

In [174]:
gdp_per_industry = gdp_per_industry.drop(0)
workforce_demographic2018 = workforce_demographic2018.drop(0)

In [178]:
workforce_demographic2018['OCC_GROUP'] = workforce_demographic2018['OCC_GROUP'].astype(str)

In [179]:
detailed_groups_df = workforce_demographic2018.loc[workforce_demographic2018.OCC_GROUP.apply(lambda cat: 'detailed' in cat)]

In [180]:
detailed_groups_raw_data = detailed_groups_df.drop(['AREA','STATE','ST','OCC_GROUP','OCC_TITLE','ANNUAL','HOURLY'], axis=1)

In [181]:
detailed_groups_raw_data.head()

,OCC_CODE,TOT_EMP,EMP_PRSE,JOBS_1000,LOC_Q,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
2,11-1011,570,5.3,0.216,0.16,95.25,"198,130",5.2,40.92,56.83,86.63,#,#,"85,110","118,210","180,190",#,#
3,11-1021,"39,200",1.9,14.958,0.95,63.42,"131,920",1.1,22.27,32.52,51.51,81.71,#,"46,310","67,650","107,140","169,970",#
4,11-1031,630,8.0,0.239,0.69,*,"57,570",4.5,*,*,*,*,*,"21,230","23,970","48,260","68,270","100,430"
5,11-2011,390,25.9,0.147,0.84,59.12,"122,970",7.1,27.10,45.76,58.58,68.07,87.05,"56,380","95,180","121,850","141,580","181,070"
6,11-2021,"3,840",6.2,1.465,0.88,81.55,"169,630",2.3,44.42,57.76,77.82,96.01,#,"92,380","120,150","161,870","199,690",#


In [182]:
detailed_groups_raw_data = detailed_groups_raw_data.replace('-','',regex=True)
gdp_per_industry = gdp_per_industry.replace('-','',regex=True)
detailed_groups_raw_data = detailed_groups_raw_data.replace(',','',regex=True)
detailed_groups_raw_data = detailed_groups_raw_data.replace('#',np.nan,regex=True)
detailed_groups_raw_data = detailed_groups_raw_data.replace('*',np.nan)
detailed_groups_raw_data = detailed_groups_raw_data.replace('**',np.nan)

In [183]:
detailed_groups_final = detailed_groups_raw_data.apply(pa.to_numeric)

In [184]:
detailed_groups_final.dtypes

OCC_CODE       int64
TOT_EMP      float64
EMP_PRSE     float64
JOBS_1000    float64
LOC_Q        float64
H_MEAN       float64
A_MEAN       float64
MEAN_PRSE    float64
H_PCT10      float64
H_PCT25      float64
H_MEDIAN     float64
H_PCT75      float64
H_PCT90      float64
A_PCT10      float64
A_PCT25      float64
A_MEDIAN     float64
A_PCT75      float64
A_PCT90      float64
dtype: object

In [185]:
from sklearn import preprocessing

x = detailed_groups_final.values #returns a numpy array   .loc[:,'TOT_EMP':]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

detailed_groups_normalized = pa.DataFrame(x_scaled)

In [186]:
detailed_groups_occ_code = pa.DataFrame() 

In [187]:
detailed_groups_normalized.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.000000,0.006237,0.108607,0.006175,0.010355,0.664933,0.664987,0.223810,0.364629,0.644684,0.878290,NaN,NaN,0.364607,0.644752,0.878329,NaN,NaN
1,0.000023,0.452414,0.038934,0.452402,0.127219,0.414106,0.414154,0.028571,0.143264,0.308586,0.474658,0.833333,NaN,0.143183,0.308649,0.474693,0.833455,NaN
2,0.000047,0.006930,0.163934,0.006871,0.088757,NaN,0.132482,0.190476,NaN,NaN,NaN,NaN,NaN,0.000057,0.018281,0.149354,0.263547,0.413381
3,0.002346,0.004158,0.530738,0.004086,0.110947,0.380221,0.380247,0.314286,0.200593,0.491636,0.555913,0.674359,0.855595,0.200651,0.491657,0.555973,0.674363,0.855607
4,0.002370,0.044006,0.127049,0.043981,0.116864,0.556974,0.557016,0.085714,0.406172,0.657542,0.777037,1.000000,NaN,0.406095,0.657648,0.777102,1.000000,NaN


In [188]:
detailed_groups_occ_code = detailed_groups_final.loc[:,:'OCC_CODE']

In [189]:
detailed_groups_normalized.iloc[:,0] = np.where(detailed_groups_normalized.iloc[:,0] >= 0, detailed_groups_occ_code.iloc[:,0], detailed_groups_normalized.iloc[:,0])

In [190]:
detailed_groups_normalized.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,111011.0,0.006237,0.108607,0.006175,0.010355,0.664933,0.664987,0.223810,0.364629,0.644684,0.878290,NaN,NaN,0.364607,0.644752,0.878329,NaN,NaN
1,111021.0,0.452414,0.038934,0.452402,0.127219,0.414106,0.414154,0.028571,0.143264,0.308586,0.474658,0.833333,NaN,0.143183,0.308649,0.474693,0.833455,NaN
2,111031.0,0.006930,0.163934,0.006871,0.088757,NaN,0.132482,0.190476,NaN,NaN,NaN,NaN,NaN,0.000057,0.018281,0.149354,0.263547,0.413381
3,112011.0,0.004158,0.530738,0.004086,0.110947,0.380221,0.380247,0.314286,0.200593,0.491636,0.555913,0.674359,0.855595,0.200651,0.491657,0.555973,0.674363,0.855607
4,112021.0,0.044006,0.127049,0.043981,0.116864,0.556974,0.557016,0.085714,0.406172,0.657542,0.777037,1.000000,NaN,0.406095,0.657648,0.777102,1.000000,NaN


In [191]:
gdp_per_industry['Occupation Code'] = gdp_per_industry['Occupation Code'].astype(float)

In [192]:
gdp_per_industry.drop(gdp_per_industry.columns.difference(['Occupation Code','Projection']), 1, inplace=True)

In [193]:
merged_inner = pa.merge(left=detailed_groups_normalized, right=gdp_per_industry, left_on=0, right_on='Occupation Code')
# In this case `species_id` is the only column name in  both dataframes, so if we skipped `left_on`
# And `right_on` arguments we would still get the same result

# What's the size of the output data?
merged_inner.shape

(651, 20)

In [194]:
merged_inner.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,Occupation Code,Projection
0,111011.0,0.006237,0.108607,0.006175,0.010355,0.664933,0.664987,0.223810,0.364629,0.644684,0.878290,NaN,NaN,0.364607,0.644752,0.878329,NaN,NaN,111011.0,1310
1,111021.0,0.452414,0.038934,0.452402,0.127219,0.414106,0.414154,0.028571,0.143264,0.308586,0.474658,0.833333,NaN,0.143183,0.308649,0.474693,0.833455,NaN,111021.0,48450
2,111031.0,0.006930,0.163934,0.006871,0.088757,NaN,0.132482,0.190476,NaN,NaN,NaN,NaN,NaN,0.000057,0.018281,0.149354,0.263547,0.413381,111031.0,750
3,112011.0,0.004158,0.530738,0.004086,0.110947,0.380221,0.380247,0.314286,0.200593,0.491636,0.555913,0.674359,0.855595,0.200651,0.491657,0.555973,0.674363,0.855607,112011.0,610
4,112021.0,0.044006,0.127049,0.043981,0.116864,0.556974,0.557016,0.085714,0.406172,0.657542,0.777037,1.000000,NaN,0.406095,0.657648,0.777102,1.000000,NaN,112021.0,3610


In [202]:
merged_inner.dropna();

In [203]:
X = np.array(merged_inner.iloc[:,1:17])
y = np.array(merged_inner.iloc[:,19:20])


In [205]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [206]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [207]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto')#algorithm='brute',metric='euclidean',algorithm='brute'

In [208]:
knn.fit(x_train, y_train);

In [209]:
predictions = knn.predict(x_test)

In [210]:
y_test_array = np.array(y_test)

In [211]:
total_sum = 0

for i in range (len(y_test)):
    distance = y_test_array[[i],:] - predictions[[i],:]
    total_sum += distance
    
total_average = abs(total_sum/len(y_test))   ## Prints the average distance between my prediction and y_train. could easily be switched to y_test - going to try for 'detailed' group
print("Average Distance between y_test and K-Nearest Neighbors predictins:", total_average)

Average Distance between y_test and K-Nearest Neighbors predictins: [[249.85321101]]


In [212]:
from sklearn.metrics import mean_squared_error    # i dont really know what this is but I think it is how far im off on my predictions, on average
mse = mean_squared_error(y_test, predictions)
rmse = mse ** (1/2)
print(rmse)

2837.4742160247915


In [213]:
from sklearn.preprocessing import scale
from sklearn import svm

In [216]:
clf = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)      # remove all the stuff inside to see what can go inside this
clf.fit(x_train, y_train.ravel());

In [217]:
predictions1 = clf.predict(x_test)

In [218]:
total_sum1 = 0

for i in range (len(y_test)):
        distance = y_test_array[[i],:] - predictions1[i]
        total_sum1 += distance
    
total_average1 = abs(total_sum1//len(y_test))
print("Average Distance between y_test and SVM predictins:",total_average1)

Average Distance between y_test and SVM predictins: [[3480]]
